In [14]:
import numpy as np
import ee
import matplotlib.pyplot as plt
import tifffile as tiff
import zipfile
import os
import requests
import io
import random

In [2]:
ee.Initialize()

In [3]:
def unzipURL(img_url, tmp_directory = None):
    """
    Downloads and unzips a file in a tmp directory.
    Hits the server once.

    img_url: A web address to a zip file.
    tmp_directory: Optional path to unzip the contents.

    Returns: Path to the directory containing the contents of the zip file.
    """

    #
    # Set up temp directory
    #
    if tmp_directory == None:
        tmp_directory="./tmp/"+str(random.randrange(99999999))
    zip_filename = os.path.join(tmp_directory, 'zipped_tile.zip')
    # make tmp directory
    if not os.path.exists(tmp_directory):
        os.makedirs(tmp_directory)
    # and delete everything in it:
    filelist = [ f for f in os.listdir(tmp_directory)]
    for f in filelist:
        os.remove(os.path.join(tmp_directory, f))

    #
    # Download file
    #
    try:
        r = requests.get(img_url)
        sc = r.status_code
    except requests.ConnectionError as e:
        raise e

    if r.status_code == 200:
        z = r.content.decode('latin-1')
        z = zipfile.ZipFile(z)
        z.extractall(tmp_directory)
        z.close()
        return tmp_directory
    elif r.status_code == 400:
        print ("Uh-oh. Status code 400 (Bad Request). Possible problems:")
        print ("- You requested a band that wasn't available in every image.")
        print ("- Some other unknown issue.")
        raise ServerError("Status code 400")
    elif r.status_code == 429:
        raise ServerError("Status code 429")
    else:
        raise ServerError("Uh-oh. Status code " + str(r.status_code))

    # Fall through on handled error/exception
    raise Exception("Unknown error in unzipURL().")


In [4]:
def tif_at_region(geCollection, resolution, bands, geo_bounds, verbose=False):
    DEFAULT_MAP_NAME = 'map_section'
    gee_img = geCollection.filterBounds(geo_bounds).select(bands).mosaic()
    bands_string = [{'id': b} for b in bands]
    path=gee_img.getDownloadUrl({
            'name': DEFAULT_MAP_NAME,
            'scale': resolution,
            'crs': 'EPSG:4326',
            'bands': bands_string,
            'region': geo_bounds.getInfo()['coordinates']
    });
    
    if verbose:
        print(path)
    
    tif_location = unzipURL(path)
    tif_filename_object = {}
    for band in bands:
        tif_filename = DEFAULT_MAP_NAME+'.'+band+'.tif'
        tif_path = os.path.join(tif_location, tif_filename)
        tif_filename_object[band] = tif_path
    return tif_filename_object

In [5]:
def img_at_region(geCollection, resolution, bands, geo_bounds, verbose=False):
    tif_band_dictionary = tif_at_region(geCollection, resolution, bands, geo_bounds, verbose)
    img_band_dictionary = {}
    for k,v in tif_band_dictionary.items():
        img_band_dictionary[k] = tiff.imread(v)
    return img_band_dictionary

In [6]:
nps_bounds = ee.Geometry.Rectangle([-121.880742, 36.600885, -121.866788, 36.594170])
tiles = img_at_region(ee.ImageCollection('USDA/NAIP/DOQQ'), 5, ['R'], nps_bounds, verbose=True)
plt.imshow(tiles['R'], cmap='gray')
plt.show()

https://earthengine.googleapis.com/api/download?docid=cb87a15197aac81b43949112106cbfb9&token=10a19dab3d7e558868d4c9d9f396e9db


ValueError: embedded null character

In [7]:
path = 'https://earthengine.googleapis.com/api/download?docid=cb87a15197aac81b43949112106cbfb9&token=10a19dab3d7e558868d4c9d9f396e9db'

In [8]:
req = requests.get(path)
status = req.status_code

In [9]:
status

200

In [10]:
req.content

b'PK\x03\x04\x14\x00\x08\x08\x08\x00\x167VM\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00map_section.R.tfwm\xcb\xc1\r\xc00\x08\x04\xc1?\xbd\x18\x1d\x98;CA\xe9\xbf\x85\xbc\xadd\x9f#myM\xf0\xa8\x12\x9c\xa3x\x16hp\\}a\xd5\xdf\xb7"\xc3\xbbq\x8a\xa9\xd8\x92m\xb9\x80\x1eP\xc9\x18\xda\x0bPK\x07\x08\xf5B\xccK<\x00\x00\x00p\x00\x00\x00PK\x03\x04\x14\x00\x08\x08\x08\x00\x167VM\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x11\x00\x00\x00map_section.R.tif\x84\xbbep\x1ck\x9e\xee\xe9\x9e\x99\xb8w6vb\xe3\xee\x87\xbd\x1b\xbb1\xdd}\xfa\x18\x04\xc5\xcc\\*13\xa3e\x81e\x0b,\xb0,f\x0b-ffff\x95\x18KR\x81\x98\x99\x19\r\x9b\x92\xec\xd3\xe7\xee\xdd\x88}UY\xf9f\x96\xa0\xfc\xab?<\xcf\x9biYY\xc9g\xff\xfe\xec\xd9\xb3\xff\xf6\xec\xd9_\xfe\xf5\xd9_\x80\x19\x1ex\xfa\xcb\xcfy"\xb0\xfd\xcb\xcf\xf9\xc3w\xfd\xeb\xcf\xf9\xc3\xf6_\xfe4\xff\xdf\xff\xf2o\xcf\xfe\x0b\xb0\xdf\x02\xb6\xff\xe3O\xe7\xff\xfb\xcf\xf9\xff\rl\xff\xe7\xcf\xef\x19\x07\xb6\xff\xfc\xd3\xf7\xa8\xffi\xfe\xbf\xf9\xff\xc7\xb3\x7f\x05\xf6

In [11]:
content = req.content

In [15]:
obj = io.BytesIO(content)

In [16]:
z = zipfile.ZipFile(obj)

In [17]:
z.extractall("./tmp/"+'pers')

In [ ]:
z = zipfile.ZipFile(z)
z.extractall(tmp_directory)
z.close()
return tmp_directory

In [ ]:
def unzipURL(url, tmp_directory=None):
    if tmp_directory == None:
        tmp_directory="./tmp/"+str(random.randrange(99999999))
    zip_filename = os.path.join(tmp_directory, 'zipped_tile.zip')
    # make tmp directory
    if not os.path.exists(tmp_directory):
        os.makedirs(tmp_directory)
    # and delete everything in it:
    filelist = [ f for f in os.listdir(tmp_directory)]
    for f in filelist:
        os.remove(os.path.join(tmp_directory, f))

    #
    # Download file
    #
    try:
        r = requests.get(img_url)
        sc = r.status_code
    except requests.ConnectionError as e:
        raise e

    if r.status_code == 200:
        z = r.content.decode('latin-1')
        z = zipfile.ZipFile(z)
        z.extractall(tmp_directory)
        z.close()
        return tmp_directory
    elif r.status_code == 400:
        print ("Uh-oh. Status code 400 (Bad Request). Possible problems:")